In [1]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx
import pickle

/tmp/ipykernel_10732/3967175050.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
local_crs = 3414

place = "singapore"

In [3]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

2023-05-20 01:50:29,854 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,856 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,859 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,864 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,867 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,870 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,873 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-05-20 01:50:29,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33755,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:35689,Total threads: 2
Dashboard: http://127.0.0.1:38401/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:38071,


In [4]:
# with open('./out/{place}/what_cells_are_in_what_cluster_dict.json', 'r') as f:
#     includes_dict = json.load(f)
    
tessellation = gpd.read_parquet(f"./out/{place}/tessellation.pq")

buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")

streets = gpd.read_parquet(f"./out/{place}/streets.pq").explode()

/tmp/ipykernel_10732/1367824222.py:8: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet(f"./out/{place}/streets.pq").explode()


In [5]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [6]:
# @dask.delayed
# def queen_contiguity_load(n1, includes_dict):
#     chunk = gpd.read_parquet(f"./out/{place}/expanded_cells_in_chunk_{int(n1)}.pq")
    
#     # weights
    # queen_1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    # queen_3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
#     return (queen_1, queen_3)

In [7]:
# file_pattern = f"./out/{place}/expanded_cells_in_chunk_*.pq"
# file_list = glob.glob(file_pattern)
# num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

# weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

# weight_out = dask.compute(weight_out)


In [8]:
# queen_1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

# queen_3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [9]:
queen_1 = libpysal.weights.fuzzy_contiguity(tessellation, tolerance=0.05, buffering=True, drop=True, buffer=20, ids="uID", silence_warnings=True)
queen_3 = momepy.sw_high(k=3, weights=queen_1)
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

/home/virgilxw/miniconda3/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 495 disconnected components.
 There are 180 islands with ids: 41, 100, 274, 275, 894, 1526, 1670, 1672, 2465, 2498, 2500, 2504, 2550, 2557, 2568, 3389, 3390, 3410, 3416, 3423, 3426, 3434, 3952, 3953, 4076, 4123, 4535, 4546, 4694, 4695, 4822, 5349, 5363, 5468, 5493, 5504, 5525, 5529, 5991, 6019, 6145, 6259, 6399, 6936, 7233, 7234, 7241, 7331, 8123, 8466, 8469, 8662, 8897, 8898, 8899, 8900, 8956, 10832, 12589, 13289, 13983, 13996, 13997, 14634, 14689, 14692, 14693, 14696, 15528, 15533, 16175, 16190, 16191, 16911, 16937, 17441, 18107, 18125, 18150, 20581, 20588, 21373, 23829, 23830, 25030, 25630, 25643, 27835, 28295, 28296, 28299, 28300, 28301, 29141, 31615, 31616, 31790, 32365, 33127, 33383, 33652, 34085, 34792, 35474, 40561, 41055, 42730, 43316, 43317, 44125, 44126, 44133, 44571, 44572, 44573, 44942, 44946, 45198, 45780, 48339, 48344

In [10]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [11]:
### Building Area
buildings['building_area'] = momepy.Area(buildings).series
tessellation['tess_area'] = momepy.Area(tessellation).series

In [12]:
buildings_future = client.scatter(buildings)
streets_future = client.scatter(streets)
tessellation_future = client.scatter(tessellation)
queen_3_future = client.scatter(queen_3)

/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/protocol/pickle.py:94: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  return pickle.loads(x, buffers=buffers)
/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/protocol/pickle.py:94: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPan

In [14]:
computations = []
### Circular Compactness
building_circular_compactness =  delayed(momepy.CircularCompactness)(buildings_future)
computations.append(building_circular_compactness)

### Elongation
building_elongation = delayed(momepy.Elongation)(buildings_future)
computations.append(building_elongation)

## Squareness
building_squareness = delayed(momepy.Squareness)(buildings_future)
computations.append(building_squareness)

building_eri = delayed(momepy.EquivalentRectangularIndex)(buildings_future)
computations.append(building_eri)

street_linearity = delayed(momepy.Linearity)(buildings_future)
computations.append(street_linearity)

# StreetProfile computation
profile = delayed(momepy.StreetProfile)(streets_future, buildings_future)
computations.append(profile)

# Tessellation computations
tess_convexity = delayed(momepy.Convexity)(tessellation_future)
computations.append(tess_convexity)

tess_neighbours = delayed(momepy.Neighbors)(tessellation_future, queen_3_future,'uID', weighted=True)
computations.append(tess_neighbours)

tess_neighbours_200 = delayed(momepy.Neighbors)(tessellation_future, dist200, 'uID', weighted=True)
computations.append(tess_neighbours_200)

tess_covered_area = delayed(momepy.CoveredArea)(tessellation_future, queen_1, "uID")
computations.append(tess_covered_area)

tess_area_iqr = delayed(momepy.Range)(
    tessellation_future, values='tess_area', spatial_weights=queen_3_future, unique_id='uID', rng=(25, 75))
computations.append(tess_area_iqr)

tess_rea_theil = delayed(momepy.Theil)(
    tessellation_future, values='tess_area', spatial_weights=queen_3_future, unique_id='uID')
computations.append(tess_rea_theil)

building_orientation = delayed(momepy.Orientation)(buildings_future)
computations.append(building_orientation)

tess_orientation = delayed(momepy.Orientation)(tessellation_future)
computations.append(tess_orientation)

building_neighbour_dist = delayed(momepy.NeighborDistance)(buildings_future, queen_1, 'uID')
computations.append(building_neighbour_dist)

building_neighbourhood_interbuilding_distance = delayed(momepy.MeanInterbuildingDistance)(buildings_future, queen_1, 'uID', 3)
computations.append(building_neighbourhood_interbuilding_distance)

In [15]:
# Compute all tasks at once using the Dask client
results = client.compute(computations)

/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 129.19 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [16]:
building_circular_compactness, building_elongation, building_squareness, building_eri, street_linearity, profile, tess_convexity, tess_neighbours, tess_neighbours_200, tess_covered_area, tess_area_iqr, tess_rea_theil, building_orientation, tess_orientation, building_neighbour_dist, building_neighbourhood_interbuilding_distance = results

In [60]:
buildings['building_circular_compactness'] = building_circular_compactness.result().series
buildings['building_elongation'] = building_elongation.result().series
buildings['building_squareness'] = building_squareness.result().series
buildings['building_eri'] = building_eri.result().series
buildings['building_orientation'] = building_orientation.result().series
streets['street_linearity'] = street_linearity.result().series
tessellation["tess_convexity"] = tess_convexity.result().series
tessellation["tess_neighbours"] = tess_neighbours.result().series
tessellation["tess_neighbours_200"] =  tess_neighbours_200.result().series
tessellation["tess_covered_area"] = tess_covered_area.result().series
tessellation["tess_area_iqr"] = tess_area_iqr.result().series
tessellation["tess_rea_theil"] = tess_rea_theil.result().series
buildings["building_orientation"] = building_orientation.result().series
tessellation["tess_orientation"] = tess_orientation.result().series
tessellation["building_neighbour_dist"] = building_neighbour_dist.result().series
tessellation["building_neighbourhood_interbuilding_distance"] = building_neighbourhood_interbuilding_distance.result().series

In [61]:
profile_result = profile.result()


streets["street_width"] = profile_result.w
streets["street_width_deviation"] = profile_result.wd
streets["street_openness"] = profile_result.o

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/frame.py", line 10775, in _reindex_for_setitem
    reindexed_value = value.reindex(index)._values
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/series.py", line 4580, in reindex
    return super().reindex(index=index, **kwargs)
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/generic.py", line 4818, in reindex
    return self._reindex_axes(
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/generic.py", line 4834, in _reindex_axes
    new_index, indexer = ax.reindex(
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 2533, in reindex
    target = MultiIndex.from_tuples(target)
  File "/home/virgilxw/.local/lib/python3.10/site-packages/pandas/core/indexes/multi.py", line 202, in new_meth
    return meth(self_or_cls, *args, **kwargs)
  File "/home/virgilxw/.local/lib/python3

In [62]:
buildings['building_weighted_circom'] = momepy.WeightedCharacter(
buildings, buildings["building_circular_compactness"], queen_3, 'uID', momepy.Area(buildings).series).series

  0%|          | 0/75039 [00:00<?, ?it/s]

In [63]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [64]:
%%time
# momepy.node_degree(primal)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [65]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [66]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [67]:
# %%time
# primal = momepy.straightness_centrality(primal)

In [68]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [69]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [70]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [71]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [72]:
cols = list(streets.columns)
cols.remove('geometry')

In [73]:
streets_dropped = streets.drop(cols,axis=1).reset_index(drop=True)

In [74]:
buildings = buildings.set_crs(local_crs)

In [76]:
# buildings=buildings.drop(["index_right"], axis=1)

In [75]:
buildings=buildings.drop(["uID_left", "index_right", "uID_right"], axis=1)

KeyError: "['uID_left' 'uID_right'] not found in axis"

In [77]:
tessellation=tessellation.drop(["uID_left", "index_right", "uID_right"], axis=1)

KeyError: "['uID_left' 'index_right' 'uID_right'] not found in axis"

In [78]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=streets_dropped, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [79]:
buildings = buildings.dropna(subset=['geometry'])
tessellation = tessellation.dropna(subset=['geometry'])

In [80]:
streets.reset_index(drop=True, inplace=True)

In [81]:
streets["nID"]= streets.index

In [82]:
buildings.drop(columns=['index_right'], inplace=True)

In [83]:
streets["nID"]= streets.index
buildings =  gpd.sjoin_nearest(buildings, streets, how="left", max_distance=1000)

In [84]:
str_align = momepy.StreetAlignment(buildings, streets,
                                   'building_orientation', 'nID',
                                   'nID').series

/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:74: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type in ["Polygon", "MultiPolygon", "LinearRing"]:
/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/distribution.py:84: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type in ["LineString", "MultiLineString"]:


In [85]:
blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'building_orientation', 'tess_orientation',
                                      'uID', 'uID')
buildings['building_cell_align'] = blg_cell_align.series

In [86]:
tessellation['tess_car'] = momepy.AreaRatio(tessellation, buildings, 'tess_area', 'building_area', 'uID').series

In [87]:
nodes = gpd.read_parquet(f"./out/{place}/vertices_frame.pq")
edges = gpd.read_parquet(f"./out/{place}/edges_frame.pq")

In [88]:
buildings = buildings.drop(["nID", "index_right", "nID"], axis=1)
nodes = nodes.drop(["vertID"], axis=1)
edges = edges.drop("edgeID", axis=1)

In [89]:
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=nodes, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [90]:
buildings = buildings.drop("index_right", axis=1)
buildings = gpd.sjoin_nearest(left_df=buildings, right_df=edges, how='left', max_distance=1000).drop_duplicates(subset=['geometry'])
duplicates = buildings[buildings.duplicated(['uID'], keep=False)]
if len(duplicates) > 0:
    print(f"There are {len(duplicates)} rows with duplicate uids.")
    print(duplicates)
else:
    print("There are no rows with duplicate uids.")


There are no rows with duplicate uids.


In [91]:
tessellation = tessellation.merge(buildings, how='left', left_on = "uID", right_on="uID" )

In [92]:
tessellation = tessellation.drop(["geometry_y", "index_right"], axis=1)
tessellation = tessellation.rename(columns={"geometry_x": "geometry"})
tessellation = gpd.GeoDataFrame(tessellation.dropna(subset=['geometry']))

In [93]:
with open(f"./out/{place}/weights.pickle", 'wb') as f:
    pickle.dump(queen_1, f)
    pickle.dump(queen_3, f)
    pickle.dump(dist200, f)
    f.close()

In [94]:
tessellation.columns=tessellation.columns.astype(str)

In [95]:
tessellation

,uID,geometry,tess_area,tess_convexity,tess_neighbours,tess_neighbours_200,tess_covered_area,tess_area_iqr,tess_rea_theil,tess_orientation,...,street_width_deviation_right,street_openness_right,betweeness_centrality_vertex_right,node_degree_right,closeness_right,global_closeness_centrality_right,global_clustering_400m_right,eigenvector_right,node_centrality_400m_right,betweeness_centrality_edge_right
0,0,"POLYGON ((3994.240 23543.420, 3944.636 23740.8...",5.857117e+05,0.778496,0.003648,0.000429,8.879329e+05,17897.582094,1.784848,42.990501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"POLYGON ((4136.802 23668.031, 4138.878 23689.8...",2.407055e+05,0.997793,0.004688,0.000852,8.603547e+05,27381.991672,1.422985,4.544136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"POLYGON ((4056.047 23767.190, 4055.080 23767.6...",3.393751e+04,0.948822,0.023931,0.002519,9.123867e+05,8029.763386,1.870591,16.579217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"POLYGON ((3933.972 23854.455, 3934.428 23860.5...",2.757822e+04,0.978591,0.029246,0.013926,6.848856e+05,17980.464103,1.827191,33.779758,...,0.0,1.0,0.0,1.0,0.0,0.000029,0.0,52156.0,8.0,0.00003
4,4,"POLYGON ((3906.695 23898.979, 3906.781 23899.5...",4.643226e+03,0.957231,0.047825,0.025051,6.748768e+04,17980.464103,1.827191,1.835183,...,0.0,1.0,0.0,1.0,0.0,0.000029,0.0,52156.0,8.0,0.00003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75034,75034,"POLYGON ((48438.627 44404.653, 48438.226 44402...",7.731463e+03,0.775146,0.006913,0.001728,2.780585e+05,54221.613877,0.196490,14.216201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75035,75035,"POLYGON ((55301.356 44330.176, 55300.829 44330...",7.336948e+05,0.992504,0.001862,0.000532,1.190246e+06,528973.183959,0.463744,37.776419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75036,75036,"POLYGON ((48934.987 44509.720, 48934.371 44510...",2.266511e+05,0.902605,0.001583,0.000000,9.090801e+05,54221.613877,0.196490,37.992753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75037,75037,"POLYGON ((52332.391 44045.472, 51961.087 43999...",3.055315e+06,0.928048,0.004516,0.000000,6.786890e+06,197473.586319,1.000884,7.346464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
tessellation.describe()

,uID,tess_area,tess_convexity,tess_neighbours,tess_neighbours_200,tess_covered_area,tess_area_iqr,tess_rea_theil,tess_orientation,building_neighbourhood_interbuilding_distance,...,street_width_deviation_right,street_openness_right,betweeness_centrality_vertex_right,node_degree_right,closeness_right,global_closeness_centrality_right,global_clustering_400m_right,eigenvector_right,node_centrality_400m_right,betweeness_centrality_edge_right
count,75039.000000,7.503900e+04,75039.000000,75039.000000,75039.000000,7.503900e+04,75039.000000,75039.000000,75039.000000,75039.000000,...,73544.000000,73544.000000,73530.000000,73530.000000,73530.000000,73530.000000,73122.000000,73530.000000,73530.000000,73544.000000
mean,37519.000000,7.343193e+03,0.925014,0.575308,0.377513,7.810045e+04,6143.107511,0.609888,21.332464,51.192885,...,1.846409,0.745807,0.001990,2.271100,0.039985,0.000069,0.057510,33866.492887,52.479723,0.001843
std,21662.037762,3.683012e+04,0.063982,0.587884,0.541751,1.644122e+05,16139.519520,0.441334,13.381529,32.380425,...,2.015880,0.251465,0.005861,0.709884,0.081186,0.000014,0.080801,20402.222546,32.604631,0.005674
min,0.000000,1.806290e+01,0.179915,0.000000,0.000000,2.553828e+02,0.000000,0.000000,0.000029,0.000000,...,0.000000,0.100000,0.000000,1.000000,0.000000,0.000029,0.000000,9.000000,0.000000,0.000000
25%,18759.500000,9.757128e+02,0.897675,0.197842,0.056833,2.560429e+04,1253.477955,0.311767,9.475269,33.953483,...,0.000000,0.541667,0.000059,2.000000,0.011679,0.000059,0.000000,17952.000000,28.000000,0.000046
50%,37519.000000,2.371945e+03,0.941525,0.372136,0.157693,4.318346e+04,3113.975216,0.491191,20.698944,41.077274,...,1.306719,0.809524,0.000284,2.000000,0.024508,0.000071,0.000000,31104.000000,46.000000,0.000244
75%,56278.500000,5.606872e+03,0.970609,0.804603,0.541313,7.462257e+04,5759.927400,0.769883,33.071616,54.842638,...,3.266565,1.000000,0.001090,3.000000,0.043666,0.000080,0.095745,48780.000000,71.000000,0.000967
max,75038.000000,3.346744e+06,1.000000,11.355089,12.869101,6.786890e+06,657679.331943,3.875544,44.998944,837.442654,...,11.813547,1.000000,0.106086,5.000000,2.089181,0.000093,0.600000,79600.000000,213.000000,0.106095


In [97]:
tessellation.to_parquet(f"./out/{place}/tessellation_stats.pq")

In [ ]:
# chunk_size = 250000
# chunks = []
# for i in range(0, len(tessellation), chunk_size):
#     chunks.append(tessellation.iloc[i:i + chunk_size])

In [ ]:
# import os
# new_directory = f"./out/{place}/tessellation/"

# if not os.path.exists(new_directory):
#     os.makedirs(new_directory)
#     print(f"Created directory: {new_directory}")
# else:
#     print(f"Directory already exists: {new_directory}")

# # iterate over all files in the directory and delete them
# for filename in os.listdir(new_directory):
#     file_path = os.path.join(new_directory, filename)
#     if os.path.isfile(file_path):
#         os.remove(file_path)
#         print(f"Deleted file: {file_path}")

In [ ]:
# for i, chunk in enumerate(chunks):
#     chunk.to_parquet(f"./out/{place}/tessellation/tesselation_chunk_{i}.pq")

In [ ]:
# duplicates = tessellation[tessellation.duplicated(['uID'], keep=False)]
# if len(duplicates) > 0:
#     print(f"There are {len(duplicates)} rows with duplicate uids.")
#     print(duplicates)
# else:
#     print("There are no rows with duplicate uids.")

In [ ]:
# import pickle

# # Open the file for reading
# with open('./out/{place}/weights.pickle', 'rb') as f:
#     # Load the variables in the same order that they were dumped
#     queen_1 = pickle.load(f)
#     queen_3 = pickle.load(f)
#     dist200 = pickle.load(f)
#     tessellation = pickle.load(f)

In [ ]:
# tessellation.to_parquet("./out/{place}/tessellation_with_data.pq", write_metadata_file=False)